# **mwsamanaga Intended Writeup** 👅🐲

(I'd like to apologize in advance for how ~forenseggs~ forensics this whole chall feels like, had to split the flag into a few pieces to allow for the multiple ideas I'd wanted to bring into the chall, resulting in this, lolol, fwiw imho this is still very misc since it's stegano+AI-heavy ...and random 💀)

I'll structure this WU like how someone would likely be approaching this w/o the knowledge, since the flow of the intended solution for this was the following, but players are pretty unlikely to get this unless they're rlly AI-oriented:
1. start at nagachat, look ard, find `REMOVE SENSITIVE` thingy interesting, noted.
2. understand config format, unzip. Look for so-called "sensitive" data, found omitted in the pkl.
3. one of the pkls has me talking abt recovering from vectors, look ard, finds vec2text.
4. use that on the vector, get Claude, noted.
5. finds pw hint and email (stegano?), `claude30!`, noted.
6. read stegano notes, gets to "perfectly secure stegano" (or honestly, could've noted from the stegano-related faiss index too), finds MEC paper.
7. MEC finds ssokota's (https://github.com/ssokota/mec), modify based on notes n some amateur hour chatgpt.
8. decrypt & decode, gets part 1.
9. NNcrypto, weird model, interesting oracle.
10. finds deployed oracle, after some head hitting wall, get to XOR conclusion
11. known-plaintext, XOR, part 2 & 3
12. yay :))

~POV: you're expecting 0 solves :kms:~ <small><small><small><small><small><small><small><small>🤷 Hey, literally no unintended yh sodara-sodara</small></small></small></small></small></small></small></small>

![ndukun](https://files.catbox.moe/g79ziu.gif)

## **Setup**

In [1]:
!sudo apt install tree

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 21 not upgraded.
Need to get 47.9 kB of archives.
After this operation, 116 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Fetched 47.9 kB in 1s (68.6 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package tree.
(Reading database ... 124926 files and directories currently instal

In [2]:
!pip install gradio-client faiss-cpu vec2text git+https://github.com/ssokota/mec.git

  Cloning https://github.com/ssokota/mec.git to /tmp/pip-req-build-6zx01bds
  Running command git clone --filter=blob:none --quiet https://github.com/ssokota/mec.git /tmp/pip-req-build-6zx01bds
  Resolved https://github.com/ssokota/mec.git to commit 72404a63529ea7e33629804d392711f4483ccf35
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.0/322.0 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB

In [3]:
PREFIXX = "/content/user/naga"

In [4]:
!wget https://files.catbox.moe/cnfukv.zip

--2025-02-20 18:07:29--  https://files.catbox.moe/cnfukv.zip
Resolving files.catbox.moe (files.catbox.moe)... 108.181.20.35
Connecting to files.catbox.moe (files.catbox.moe)|108.181.20.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5185152 (4.9M) [application/zip]
Saving to: ‘cnfukv.zip’

cnfukv.zip          100%[===================>]   4.94M  4.17MB/s    in 1.2s    

2025-02-20 18:07:31 (4.17 MB/s) - ‘cnfukv.zip’ saved [5185152/5185152]



In [5]:
!7z x -parsipbuatara cnfukv.zip -y


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,12 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (50657),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 5185152 bytes (5064 KiB)

Extracting archive: cnfukv.zip
--
Path = cnfukv.zip
Type = zip
Physical Size = 5185152

  0%    Everything is Ok

Folders: 37
Files: 57
Size:       5288014
Compressed: 5185152


# **Reconnaissance**

To start off, players are prolly gonna look at the directory structure.  

In [6]:
!tree user

user
└── naga
    ├── notes
    │   ├── 011024_files
    │   │   └── testing.md
    │   ├── 011024.md
    │   ├── 22323.md
    │   └── 30323.md
    ├── personal
    │   ├── beloved.png
    │   ├── gawkgawkgawk.png
    │   ├── hintpw
    │   └── pwmail.txt
    └── projects
        ├── Nagachat
        │   ├── main.py
        │   ├── savedstuff
        │   │   ├── dev_081124.llchat
        │   │   └── dev_150124.llchat
        │   └── TODO.md
        └── nncrypto
            ├── app.py
            ├── rajaKripto.safetensors
            ├── README.md
            └── requirements.txt

8 directories, 16 files


## Directory Structure
So the idea of this structure was, players would likely check out the immediate and less wacky (i.e., less reading someone's diary n more code) code stuff first, so `projects/Nagachat/main.py` and `projects/nncrypto/main.py`.

Opening up Nagachat tho, will prolly get, just, barely any understanding of the problem, it's a pretty complex piece of code, so it's likely they'll leave it for the moment being atm. Which is why we're actually going for part 2 & 3 first most likely.

# **NNCrypto (part 2 & 3)**
So, moving to `app.py`, seems simple enough, feels like an oracle ~a shitty one~, they might try to either:  
  a. Immediately crack the thing (in which this is effectively just an XOR btw, crypto players pls don't roast me 😭)
```
    def _scramble(self, x, k):
        _m = 0.5 * (torch.tanh(10 * (x - 0.5)) + 1)
        _n = k.round()
        return (_m - _n).abs().clamp(0, 1)

    def encode(self, x, k):
        _t = self._e(x)
        _v = self._f1(_t)
        _p = torch.sigmoid(_v)
        _k = k.unsqueeze(1).repeat(1, _p.size(1), 1)
        return self._scramble(_p, _k)
```  

  b. Or, and imho this is arguably the more likely scenario here, they'll check out *what* this is even used in, so either by digging into notes, or the git, which yields the hf spaces link.

Then begins the cracking :)



## Cracking

Once the XOR nature is realized, the cracking isn't actually that hard.

You just, need to figure out how to turn strings into the tensors we're looking for, and the base64 thing.

I've provided the utils in the gradio code for players' convenience, but with a bit of claude/chatgpt/deepseek/whateverflavorofai and prayers, *hopefully* you can get some scripting going.

In [7]:
import torch
from safetensors.torch import load_model, safe_open
import base64
import numpy as np
from pathlib import Path
from gradio_client import Client
import random
import requests
import json
import torch.nn as nn
from google.colab import userdata

MODEL_URL = "https://files.catbox.moe/6yulot.safetensors"
MODEL_PATH = Path("/content/user/naga/projects/nncrypto/rajaKripto.safetensors")

device = "cpu"

class RajaKripto(nn.Module):
    def __init__(self, vocab_size, hidden_dim=256, char_to_idx=None, idx_to_char=None):
        super().__init__()
        self._e = nn.Embedding(vocab_size, hidden_dim)
        self._f1 = nn.Linear(hidden_dim, hidden_dim)
        self._f2 = nn.Linear(hidden_dim, hidden_dim)
        self._f3 = nn.Linear(hidden_dim, vocab_size)
        self._dim = hidden_dim

        if char_to_idx and idx_to_char:
            self.init_dicts(char_to_idx, idx_to_char)

    def init_dicts(self, char_to_idx, idx_to_char):
        self.register_buffer('_char_to_idx_keys', torch.tensor([ord(c) for c in char_to_idx.keys()], dtype=torch.long))
        self.register_buffer('_char_to_idx_values', torch.tensor(list(char_to_idx.values()), dtype=torch.long))
        self.register_buffer('_idx_to_char_keys', torch.tensor(list(idx_to_char.keys()), dtype=torch.long))
        self.register_buffer('_idx_to_char_values', torch.tensor([ord(c) for c in idx_to_char.values()], dtype=torch.long))

    @property
    def char_to_idx(self):
        return {chr(k.item()): v.item() for k, v in zip(self._char_to_idx_keys, self._char_to_idx_values)}

    @property
    def idx_to_char(self):
        return {k.item(): chr(v.item()) for k, v in zip(self._idx_to_char_keys, self._idx_to_char_values)}

    def _scramble(self, x, k):
        _m = 0.5 * (torch.tanh(10 * (x - 0.5)) + 1)
        _n = k.round()
        return (_m - _n).abs().clamp(0, 1)

    def encode(self, x, k):
        _t = self._e(x)
        _v = self._f1(_t)
        _p = torch.sigmoid(_v)
        _k = k.unsqueeze(1).repeat(1, _p.size(1), 1)
        return self._scramble(_p, _k)

    def decode(self, x, k):
        _k = k.unsqueeze(1).repeat(1, x.size(1), 1)
        _d = self._scramble(x, _k)
        _h = torch.relu(self._f2(_d))
        return self._f3(_h)

    def forward(self, x, k, decrypt=False):
        return self.decode(x, k) if decrypt else self.encode(x, k)

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(69)

def key_to_text(key_tensor):
    if key_tensor.dim() > 1:
        key_tensor = key_tensor.squeeze(0)

    key_bits = ''.join([str(int(bit.item())) for bit in key_tensor])

    recovered_bytes = []
    for i in range(0, len(key_bits), 8):
        byte_bits = key_bits[i:i+8]
        if len(byte_bits) == 8:
            byte_val = int(byte_bits, 2)
            recovered_bytes.append(byte_val)

    try:
        recovered_text = bytes(recovered_bytes).decode('utf-8')
        return recovered_text
    except UnicodeDecodeError:
        return "Unable to decode key to valid UTF-8 text"

def b64_to_tensor(b64_str):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    combined = base64.b64decode(b64_str.encode('utf-8'))
    shape_bytes = combined[:8]
    data_bytes = combined[8:]
    shape_info = np.frombuffer(shape_bytes, dtype=np.int32)
    bits = np.unpackbits(np.frombuffer(data_bytes, dtype=np.uint8))
    return torch.tensor(bits, dtype=torch.float, device=device).reshape(1, shape_info[0], shape_info[1])

def text_to_tensor(text, char_to_idx, device=None):
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    return torch.tensor([char_to_idx.get(c, 0) for c in text], dtype=torch.long, device=device)

def ensure_model():
    if not MODEL_PATH.exists():
        print("Downloading model...")
        response = requests.get(MODEL_URL)
        MODEL_PATH.write_bytes(response.content)

    with safe_open(MODEL_PATH, framework="pt") as f:
        metadata = f.metadata()
        char_to_idx = {k: int(v) for k, v in json.loads(metadata["char_to_idx"]).items()}
        idx_to_char = {int(k): v for k, v in json.loads(metadata["idx_to_char"]).items()}

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = RajaKripto(len(char_to_idx)).to(device)
    model.init_dicts(char_to_idx, idx_to_char)
    load_model(model, str(MODEL_PATH))
    return model

def query_oracle(text, client):
    """Get encryption from Gradio endpoint"""
    result = client.predict(
        text,
        api_name="/encrypt_interface"
    )
    return result

def attack(gradio_url):
    # Setup Gradio client
    client = Client(gradio_url, hf_token=userdata.get("HF_KEY"))

    # Load model for computing binary_x
    model = ensure_model()

    b64_encrypted_text = client.predict(
        api_name="/geeHMM"
    )

    print(f"Encrypted text (base64): {b64_encrypted_text}")

    # 1. Choose our plaintext
    chosen_text = "A" * 21

    # 2. Get its encryption from the oracle
    encrypted_b64 = query_oracle(chosen_text, client)
    encrypted = b64_to_tensor(encrypted_b64)

    # 3. Get x
    with torch.no_grad():
        tensor = text_to_tensor(chosen_text, model.char_to_idx).unsqueeze(0)
        x = model._e(tensor)
        x = model._f1(x)
        x = torch.sigmoid(x)
        binary_x = 0.5 * (torch.tanh(10 * (x - 0.5)) + 1)
        binary_x = binary_x.round() # Round to address numerical instability

    # 4. Recover key through XOR, (x ⊕ k) ⊕ x = k
    recovered_key = (binary_x[0,0].to(torch.int8) ^ encrypted[0,0].to(torch.int8)).float()

    print("\nRecovered key bits (first 32):")
    print(recovered_key[:32].tolist())

    # 5. Decrypt target ciphertext
    target_encrypted_tensor = b64_to_tensor(b64_encrypted_text)
    with torch.no_grad():
        decrypted = model.decode(target_encrypted_tensor, recovered_key.unsqueeze(0))
        decoded_text = "".join([model.idx_to_char[idx.item()] for idx in decrypted.argmax(dim=-1)[0]])

    return recovered_key, decoded_text

if __name__ == "__main__":
    GRADIO_URL = "spuun/mwsamanaga"

    recovered_key, decrypted_text = attack(GRADIO_URL)
    print(f"Decrypted text: {decrypted_text}")
    # print(f"\nRecovered key: {key_to_text(recovered_key)}")
    print("\nAttack complete!")

Loaded as API: https://spuun-mwsamanaga.hf.space ✔
Encrypted text (base64): GAAAAAABAABq4SqddFyJSd95+7PbIbC8ucYbYyZ9nmz5eBuG636eQiRhKpFSSABxdvD9TloXkfYTdLH2c24dyucIjhR3xQ5eTiQj2zLphQ8R3LXzXrfMGWNVIIHbefXGuzlT9OeR71Nq4SqddFyJSd95+7PbIbC8ucYbYyZ9nmz5eBuG636eQr5Naz8sQJFnn63X4R4lvBgDRan30x4OBmBIO9hvwa1bfVZtHThe1EJu+92CFhUTAIygFzLiJkXuhAJ9DeoI/276KXu6VOi8Rd/eMTrwdLoyUEQzCdJ9aO6HcNpc/Vb4fmrhKp10XIlJ33n7s9shsLy5xhtjJn2ebPl4G4brfp5CfiEr+TnUAWzb4XX6qNPOFZ4EOrPwXs/4u0F9iKFE/rn6KXu6VOi8Rd/eMTrwdLoyUEQzCdJ9aO6HcNpc/Vb4fsuYG2+cXQlOWvnmngjjhmCvzFiypl5txu5JXmBsWu2WxHKw3u3v9WNenDS4VBPLqczlJA7C/z/u0H0ejKFgAgiKICvV9vqAuR/9ZzBanL0ynGQRI3ZjrY7jIP6E9sGPBn1WbR04XtRCbvvdghYVEwCMoBcy4iZF7oQCfQ3qCP9ufVZtHThe1EJu+92CFhUTAIygFzLiJkXuhAJ9DeoI/259Vm0dOF7UQm773YIWFRMAjKAXMuImRe6EAn0N6gj/buR9allszUrlZNmZjF2xwzWI8RKTw79a6qUBdX/rIX9Y5H1qWWzNSuVk2ZmMXbHDNYjxEpPDv1rqpQF1f+shf1jkfWpZbM1K5WTZmYxdscM1iPESk8O/WuqlAXV/6yF/WGrhKp10XIlJ33n7s9shsLy5xhtjJn2ebPl4G4brfp5C/c3j+P0sSepf4TEKLDmWYAxgGTwXliBm6ApK3s5B8XGKICvV9vqAuR/9ZzBa

Now, idk if you're gonna immediately think to decode that key, but if you saw the text_to_key function, I'd like to think that this is a pretty natural progression, so:

In [8]:
print(f"\nRecovered key: {key_to_text(recovered_key)}")


Recovered key: nah_lmao_pt3_3011d003e}nah_lmao_


So that's 2 and 3 DONE 🎉

# **Nagachat (part 1)**

This is the part where the "guessiness" might get me most roasted for, so um, here goes 💀

## Read TODO.md

To start off, what's... "unlikely", but maybe, ~it might not be *my* first thought at least~ is to read the TODO.md, this'd be a pretty significant bootstrap for the search, since you'll likely go "oOoOoo, sensitive info? Index?", and then... idk, look for what this "index" is.

In [9]:
!cat $PREFIXX/projects/Nagachat/TODO.md

# TODOS
[x] Functional generation  
[x] User interface  
[x] COnfig saving  
[x] Config loading
[x] ratelimiting (to be tested)  
[] better UI (ongoing)    
[x] FAISS  
[x] Nice unrestricted model (decided on https://huggingface.co/mradermacher/L3.2-JametMini-3B-MK.III-GGUF)
[x] RAG  
[] Better embedding models???  
[] Better rAG?? (CoT??)  
[] REMOVE AND OMIT SENSITIVE SHIT FROM THE INDEXES (ongoing)


## Extracting from .llchat

After reading the code a bit, the players will come to the conclusion that .llchat is really just, a zip file. So, extraction of all of these it is...

In [10]:
!rm -r $PREFIXX/projects/Nagachat/savedstuff/*/

rm: cannot remove '/content/user/naga/projects/Nagachat/savedstuff/*/': No such file or directory


In [11]:
!ls $PREFIXX/projects/Nagachat/savedstuff

dev_081124.llchat  dev_150124.llchat


In [12]:
import os
import subprocess

folder = os.path.join(PREFIXX, 'projects/Nagachat/savedstuff')
for f in os.listdir(folder):
    if f.endswith('.llchat'):
        dirname = os.path.splitext(f)[0]
        full_path = os.path.join(folder, f)
        out_dir = os.path.join(folder, dirname)
        os.makedirs(out_dir, exist_ok=True)
        subprocess.run(['7z', 'x', full_path, f'-o{out_dir}'])

In [13]:
!ls $PREFIXX/projects/Nagachat/savedstuff

dev_081124  dev_081124.llchat  dev_150124  dev_150124.llchat


## Wat to do

From this, it's concluded that "index" in the TODO.md refers to FAISS indices.

Digging into the states of these two indices, will yield:  
a. Existence of some "perfectly secure steganography".  
b. Some omission of... heros? Might make tangential connection to what's been said in TODO.md.  
c. text embedding inversion?  
~d. Cute Naga bot :33 :pepega:~

Now, players could go, literally in any of these three directions, but imho the one that makes the most sense to me (assuming they even make the connection to recovery of the hero text), is to grab the faiss, extract, do what c says. So this is where we're going next.



## Extract vector's "preimage"

Looking up "text embedding inversion" will likely get you to Morris et al.'s paper first (Morris et al. _"Text Embeddings Reveal (Almost) As Much As Text"_. (2023)), since it's practically the foundational paper on this topic afaik, most papers to date reference this when talking abt text embedding inversion. One of the best ones by empirical results imho at least lol.

Anw, going briefly over it, when we talk about text embeddings, it's essentially like hashing, where there's a bottleneck structure into a fixed n-dimensional vector  
![diagram](https://files.catbox.moe/0tnkid.png)  
which implies the loss of some level of information fidelity resulting from the pigeonhole principle. What Morris et al.'s paper implies from their empirical results is that, this bottleneck isn't as bad as something like hashing, where the loss of information is significant.

*If*, say, an adversary only has the vectors to some sensitive topic, turns out, it's extractable, unlike hashes. This is an issue, because if you're looking to make some local app, and somehow the adversary manages to recover even just the vector, you're risking users' data via this vector.

In [14]:
import pickle
import os

pkl = pickle.load(open(os.path.join(PREFIXX, "projects/Nagachat/savedstuff/dev_081124/memory_state.pkl"), "rb"))

In [15]:
pkl["texts"][0]

'My hero is <REDACTED, SENSITIVE DATA>'

In [16]:
import faiss

index = faiss.read_index(os.path.join(PREFIXX, "projects/Nagachat/savedstuff/dev_081124/memory_index.faiss"))

vector = torch.tensor([index.reconstruct(0)])

<ipython-input-16-de98674e5615>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  vector = torch.tensor([index.reconstruct(0)])


Anyway, got to Morris et al.'s paper, got their repo, got vec2text, set it up, get it running on that vector.

In [17]:
import vec2text
import torch
from transformers import AutoModel, AutoTokenizer, PreTrainedTokenizer, PreTrainedModel

encoder = AutoModel.from_pretrained("sentence-transformers/gtr-t5-base").encoder.to("cuda")
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/gtr-t5-base")
corrector = vec2text.load_pretrained_corrector("gtr-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/219M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

...

pytorch_model-00004-of-00006.bin:   0%|          | 0.00/198M [00:00<?, ?B/s]

pytorch_model-00005-of-00006.bin:   0%|          | 0.00/187M [00:00<?, ?B/s]

pytorch_model-00006-of-00006.bin:   0%|          | 0.00/37.8M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/69.0 [00:00<?, ?B/s]

In [18]:
vec2text.invert_embeddings(
    embeddings=vector.cuda(),
    corrector=corrector,
    num_steps=20,
)

['My hero is           Claude Elwood Shannon ']

Oki, so that's where Claude comes in. Now, after some digging, we'll find:

In [19]:
!cat $PREFIXX/personal/pwmail.txt

Subject: Research Data Security Protocol Update

From: security@nagai.ai
To: research-team@nagai.ai

Dear Research Team,

To ensure protection of all research materials and data (including encoded files, steganography outputs, and experimental results), please follow these standardized security protocols:

1. All research outputs must use a protection key following this format:
   - One reference name (lowercase)
   - Two digits
   - One special character from: !@#$%

Format: name##symbol

Choose a reference name that's personally meaningful - many researchers use historical figures in their field.

Apply this format for ALL research data protection operations.

Best regards,
Security Team
NagAI Research Division


In [20]:
!cat $PREFIXX/personal/hintpw

forgot pw hint: your hero, birth date, !


Formatting, gets `claude30!`

## Minimum-Entropy Coupling

So we'd get to this from the "perfectly secure steganography" hint, then googling. Atm, this consistently gets you to MEC, see:
![image](https://files.catbox.moe/4hy374.jpg)

I'm not gonna get too detailed into the maths behind what this thingy does, you can read up on the paper on that, but essentially, MEC tries to minimize the entropy of the joint distribution which couples the logits sequence with the secret's, such that, there's this coupling which allows you to determine what's the most likely byte given a token at a given sequence position (this is oversimplified lol)

No get? Here's *very* rough minimal explanation, which, isn't 1-to-1 to what the FIMEC thing does, but fwiw it helped me wrap my head ard the thing, at least:
```
# Secret: equal chance of 0 or 1
secret_dist = [0.5, 0.5]  

# Cover: "Hello" appears 70% of time, "Hi" 30% in normal text
cover_dist = [0.7, 0.3]   

# MEC might create a coupling like this:
coupling = {
    # if secret is 0:
    0: {"Hello": 1.0, "Hi": 0.0},    # Always output "Hello"
    # if secret is 1:
    1: {"Hello": 0.4, "Hi": 0.6}     # Output "Hi" more often
}

# So now:
# When you see "Hello" -> probably secret was 0
# When you see "Hi" -> definitely secret was 1

# And overall:
# "Hello" still appears 70% of time (0.5 * 1.0 + 0.5 * 0.4 = 0.7)
# "Hi" appears 30% of time (0.5 * 0.0 + 0.5 * 0.6 = 0.3)
# So it maintains the natural frequencies!

# The "minimum" here comes from the coupling which ensures minimal "ambiguity", so

# BAD coupling (high entropy/uncertainty):
# bad_coupling = {
#    0: {"Hello": 0.7, "Hi": 0.3},    
#    1: {"Hello": 0.7, "Hi": 0.3}     
# }
# Problem: If you see "Hello", no idea if secret was 0 or 1!
# High entropy = high uncertainty in recovery

# GOOD coupling (low entropy):
# good_coupling = {
#    0: {"Hello": 1.0, "Hi": 0.0},    
#    1: {"Hello": 0.4, "Hi": 0.6}     
# }
# Better:
# - See "Hello" -> probably 0
# - See "Hi" -> definitely 1
# Low entropy = more certainty in recovery
```

~Side note, personally, I think this is a bold claim fwiw lol, but the logic lines up ok imho as the marginals are *technically* indistinguishable practically-speaking~

The idea was, you go over to the paper, conclude that it is MEC, then, see this

In [21]:
!cat $PREFIXX/notes/011024.md

Went back on that perfectly secure idea, interesting stuff, paper did a code release, found an implementation on github alrd adapting it n some other derivatives for easier stuff on logits. 

Had to modify the initialization a bit, since BOS and shit wacky, snippet for future ref (note 4 future me: You'll need to modify the marginal and stuff as well to work with this, also that "\n" in marginal's __init__ is... weird. I got rid of it):
```python
class LLM:
    def __init__(self, model_name: str = "gpt2"):
        """Initialize language model and tokenizer.

        Args:
            model_name: Name of the model from HuggingFace hub.
        """
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name, device_map=self.device)
        self.vocab_size = self.model.config.vocab_size
        self.vocab = np.array(
            [se

> Went back on that perfectly secure idea, interesting stuff, paper did a code release, __*found an implementation on github alrd adapting it n some other derivatives for easier stuff on logits.*__

From there, you get to ssokota's MEC implementation
https://github.com/ssokota/mec (fwiw i think this could end up a softlock since some ppl might end up looking at the code release, but... tis what it is :kms:)

From there, you get

In [22]:
!cat $PREFIXX/notes/011024_files/testing.md

Setup:
```
model = LLMManager.get_model(
    "Hastagaras/L3.2-JametMini-3B-MK.III",
)

covertext_dist = LLMMarginal(
    prompt="Alan Turing was",
    max_len=128,
    temperature=1.0,
    k=64,
    lm_model=model
)

from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC

def derive_key(shared_secret, salt, length):
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=length,
        salt=salt,
        iterations=100000,
    )
    return kdf.derive(shared_secret.encode())

bytetext = plaintext.encode("utf-8")

cipher_len = len(bytetext) # Should be 22
key_salt = b'salting'
shared_private_key = derive_key(shared_secret, key_salt, cipher_len)
assert len(bytetext) == cipher_len
```

Output:
```
"Alan Turing was" + "---SEP---" + stego_decoded
# Alan Turing was---SEP--- a mathematician and computer scientist famous for developing the first practical computer, the Turing Machine. He also proposed the Turing Test, 

Here, you can infer that it's:  
a. Encrypted, in some way  
b. Gonna need some mods to the existing ssokota's code to work with my setup (which uses Hastagras' custom Jamet model)

## FIMEC Setup

Looking ard, reading in the examples, you'll find an "encrypted" example from ssokota's thing, which uses FIMEC, so you can just, follow their setup here.

In [23]:
from mec import FIMEC

You can grab this from https://github.com/ssokota/mec/blob/master/examples/random_string.py

In [24]:
import numpy as np

class RandomString:
    def __init__(self, num_chars: int, string_len: int):
        """A FactoredMarginal that generates uniform random strings.

        Args:
            num_chars: Number of characters in the alphabet.
            string_len: Length of the generated strings.

        Attributes:
            num_chars: Number of characters in the alphabet.
            string_len: Length of the generated strings.
            component_distributions: Attribute from FactoredMarginal protocol.
            ll: Log-likelihood of states.
        """
        self.num_chars = num_chars
        self.string_len = string_len
        self.component_distributions = [
            np.ones(num_chars) / num_chars for _ in range(string_len)
        ]
        self.ll = self.string_len * np.log(1 / self.num_chars)

    def evaluate(self, x: list[int]) -> float:
        """Implements method from `SupportsEvaluate` protocol."""
        return self.ll

    def sample(self) -> tuple[list[int], float]:
        """Implements method from `SupportsSample` protocol."""
        seq = np.random.choice(range(self.num_chars), size=self.string_len).tolist()
        return seq, self.ll

## LLM Setup

Looking at it, it's p obv that this isn't gonna b able to use the encrypted example 1-to-1, since
```
model = LLMManager.get_model(
    "Hastagaras/L3.2-JametMini-3B-MK.III",
)

covertext_dist = LLMMarginal(
    prompt="Alan Turing was",
    max_len=128,
    temperature=1.0,
    k=64,
    lm_model=model
)

from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC

def derive_key(shared_secret, salt, length):
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=length,
        salt=salt,
        iteratiaons=100000,
    )
    return kdf.derive(shared_secret.encode())

bytetext = plaintext.encode("utf-8")

cipher_len = len(bytetext)
key_salt = b'salting'
shared_private_key = derive_key(shared_secret, key_salt, cipher_len)
assert len(bytetext) == cipher_len
```
So you'll have to do some GPT shenanigans here to get it to adapt to work with this and my LLM class, but I've left enough to *hopefully* allow for enuf leeway to allow you to get it up and running.

In [25]:
from typing import List, Optional
import torch
import torch.nn as nn
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM

class LLM:
    def __init__(self, model_name: str = "gpt2"):
        """Initialize language model and tokenizer.

        Args:
            model_name: Name of the model from HuggingFace hub.
        """
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name, device_map=self.device)
        self.vocab_size = self.model.config.vocab_size
        self.vocab = np.array(
            [self.tokenizer.decode([i]) for i in range(self.vocab_size)]
        )
        self.model.to(self.device)

    def conditional(self, text: str, temperature: float) -> torch.Tensor:
        """Compute conditional distribution given text."""
        with torch.no_grad():
            encoded_input = self.tokenizer(
                text,
                return_tensors="pt",
            ).to(self.device)
            outputs = self.model(encoded_input["input_ids"])
            logits = outputs[0][:, -1, :] / temperature
            return nn.Softmax(dim=-1)(logits)

    def top_k_conditional(self, text: str, temperature: float, k: int) -> np.ndarray:
        """Compute top-k conditional distribution given text."""
        conditional = self.conditional(text, temperature)
        conditional = conditional.to(dtype=torch.float32)
        kth = torch.topk(conditional, k).values.flatten()[-1]
        conditional[conditional < kth] = 0
        conditional /= conditional.sum()
        return conditional.cpu().numpy().reshape(-1)

    def reduced_ids(self, prompt: str, text: str, k: int) -> Optional[List[int]]:
        """Token IDs using indexing post top-k reduction."""
        prompt_tokens = self.tokenizer(prompt)["input_ids"]
        text_tokens = self.tokenizer(text)["input_ids"]
        if text_tokens[0] == self.tokenizer.bos_token_id:  # Skip BOS token
          text_tokens = text_tokens[1:]

        reduced_tokens = []

        for i, token in enumerate(text_tokens):
            inputs = torch.tensor([prompt_tokens + text_tokens[:i]]).to(self.device)
            outputs = self.model(inputs)[0][0, -1, :]
            top_k_idx = outputs >= torch.topk(outputs, k).values.flatten()[-1]
            if top_k_idx[token]:
                correct_idx = torch.arange(outputs.shape[-1], device=self.device)[top_k_idx] == token
                assert correct_idx.sum() == 1
                reduced_tokens.append(correct_idx.nonzero(as_tuple=True)[0].item())
            else:
                print(f"Failed at position {i}")
                print(f"Token: {self.tokenizer.decode([token])}")
                print(f"Top k tokens: {self.tokenizer.decode(torch.where(top_k_idx)[0].tolist())}")
                return None
        return reduced_tokens

Grabbing from https://github.com/ssokota/mec/blob/master/examples/gpt2.py, you can modify to work w/ a custom LLM and get rid of the newline at prompt

In [26]:
import numpy as np

class GPT2Marginal:
    def __init__(
        self,
        prompt: str,
        max_len: int,
        temperature: float,
        k: int,
        lm_model
    ):
        """GPT2 Autoregressive Marginal.

        Args:
            prompt: Prompt to condition on.
            max_len: Maximum length of the generated text.
            temperature: Temperature for sampling.
            k: Number of top elements to consider.

        Attributes:
            max_len: Maximum length of the generated text.
            temperature: Temperature for sampling.
            k: Number of top elements to consider.
            branching_factor: Attribute from `AutoRegressiveMarginal` protocol.
            lm_model: GPT-2 model.
            prompt: Prompt to condition on.
            mapping: Mapping from prefixes to feasible tokens.

        Raises:
            ValueError: If prompt is empty.
        """
        if len(prompt) == 0:
            raise ValueError
        self.max_len = max_len
        self.temperature = temperature
        self.k = k
        self.branching_factor = k
        self.lm_model = lm_model
        # Add newline to prompt to separate it from the query text.
        self.prompt = prompt #+ "\n"
        self.mapping: dict[tuple[int, ...], np.ndarray] = {}

    def conditional(self, prefix: list[int]) -> np.ndarray:
        """Implement method from `AutoRegressiveMarginal` protocol."""
        decoded_text = self.decode(prefix)
        conditional = self.lm_model.top_k_conditional(
            self.prompt + decoded_text, self.temperature, self.k
        )
        mask = conditional > 0
        self.mapping[tuple(prefix)] = np.arange(self.lm_model.vocab_size)[mask]
        return conditional[mask]

    def evaluate(self, prefix: list[int]) -> float:
        """Implement method from `SupportsEvaluate` protocol."""
        ll = 0
        for upper in range(len(prefix)):
            conditional = self.conditional(prefix[:upper])
            ll += log(conditional[prefix[upper]])
        return ll

    def is_terminal(self, prefix: list[int]) -> bool:
        """Implement method from `AutoRegressiveMarginal` protocol."""
        if len(prefix) > self.max_len:
            raise ValueError
        return len(prefix) == self.max_len

    def encode(self, text: str) -> list[int] | None:
        """Encode given into coupling representation.

        Args:
            text: Text to encode.

        Returns:
            Encoded prefix, None if encoding fails.
        """
        return self.lm_model.reduced_ids(self.prompt, text, self.k)

    def decode(self, prefix: list[int]) -> str:
        """Decode given prefix.

        Args:
            prefix: Prefix to decode.

        Returns:
            Decoded text.
        """
        decoded_tokens = []
        for k, z_i in enumerate(prefix):
            if tuple(prefix[:k]) not in self.mapping:
                self.conditional(prefix[:k])
            decoded_tokens.append(self.mapping[tuple(prefix[:k])][z_i])
        decoded_text = self.lm_model.tokenizer.decode(decoded_tokens)
        return decoded_text

    def sample(self) -> tuple[list[int], float]:
        """Implement method from `SupportsSample` protocol."""
        prefix: list[int] = []
        likelihoods: list[float] = []
        while len(prefix) < self.max_len:
            conditional = self.conditional(prefix)
            z_k = np.random.choice(range(len(conditional)), p=conditional)
            prefix.append(z_k)
            likelihoods.append(conditional[z_k])
        return prefix, np.log(likelihoods).sum()

## Sooolveeeeee

Here's what I got up and running after a bit of back and forth w/ Claude, fwiw this isn't the "intended", but I tried to repro what I had [here](https://colab.research.google.com/drive/1gLiDL_i3tpwWxsjA87SZnBudJ43-6kGM?usp=sharing) w/o, like, literally just, reimplementing anything and everything in ssokota's lib.

In [27]:
# Setup LLM
model = LLM(model_name="Hastagaras/L3.2-JametMini-3B-MK.III")

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

In [28]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch.nn as nn
import numpy as np
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from mec.utilities import log

# The stegotext we received
received_text = """Alan Turing was---SEP--- a mathematician and computer scientist famous for developing the first practical computer, the Turing Machine. He also proposed the Turing Test, for which the Turing Test is named. He was also a key figure in cracking the German Enigma code during World War II. But he was caught up in a criminal law by the British government and "chemically castrated." He died in 1954 at the age of 41. He was posthumously pardoned for his conviction in 2009. The Alan Turing Act in 2009 also overturned the conviction of many others who were convicted for the same crimes he was convicted for. His"""

# Split into prompt and stegotext
prompt, stegotext = received_text.split("---SEP---")

def derive_key(shared_secret, salt, length):
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=length,
        salt=salt,
        iterations=100000,
    )
    return kdf.derive(shared_secret.encode())

# Setup distributions
covertext_dist = GPT2Marginal(
    prompt=prompt,
    max_len=128,
    temperature=1.0,
    k=64,
    lm_model=model
)

# Get encoded stegotext tokens
encoded_stegotext = covertext_dist.encode(stegotext)
if encoded_stegotext is None:
    print("Failed to encode stegotext!")
    exit()

# Setup RandomString for ciphertext distribution
cipher_len = 25
ciphertext_dist = RandomString(num_chars=2**8, string_len=cipher_len)

# Create FIMEC instance
imec = FIMEC(ciphertext_dist, covertext_dist)

# Try to recover the ciphertext
estimated_ciphertext, _ = imec.estimate_x_given_y(encoded_stegotext)

# Decrypt using shared secret
key_salt = b'salting'
shared_secret = "claude30!"
shared_private_key = derive_key(shared_secret, key_salt, cipher_len)

# Try to recover the hidden message
estimated_bytetext = bytes([a ^ b for a, b in zip(estimated_ciphertext, shared_private_key)])

try:
    recovered_message = estimated_bytetext.decode("utf-8")
    print("Recovered message:", recovered_message)
except UnicodeDecodeError:
    print("Failed to decode - byte sequence:", estimated_bytetext)

Recovered message: ARA6{no_unintended_yh_pt1


# **Final Thoughts** 🙃🔫

The following is some of my personal thoughts on this chall, if ur just here for the WU u don't have to read this.

Mmmm, well, when i initially conceptualized this chall, the idea was to incorporate the MEC stegano no matter what, since it was a very interesting concept and seemed to b promising big stuff (so I wanted to bring it to light for players).

Problem was, it's **_too_** AI-heavy askdbaks 😭 so there's the possibility that, it feels more like... a research paper rather than, yknow, an actual PoC, so to *hopefully* keep the engagement, I added that crypto part. My fear here is that, this ended up *way* too big for what the timeframe might permit you to explore.

I'd wanted to flesh out the concept more so the transition to understanding the MEC was smoother, but the timeframe I was provided w/ didn't really permit that, and I had other stuff to handle outside of this which really didn't allow me to allocate 100% of my efforts just on this chall.

So um, if this chall ended up feeling a bit too guessy and/or too... "wtf?" :kms: for you, I sincerely apologize ehehe 🙇